<a href="https://colab.research.google.com/github/dhkdsns20/AI-study/blob/main/Ch_5_Tree_Algorithm(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ch.5 트리 알고리즘 (2)교차 검증과 그리드 서치

In [2]:
"""### 교차 검증과 그리드 서치"""

"""## 검증 세트"""
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')



data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()



from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)



sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)


print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [3]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [4]:
"""## 교차 검증"""
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00953054, 0.00841188, 0.00874305, 0.00897098, 0.00831723]), 'score_time': array([0.0013473 , 0.00125837, 0.00124693, 0.00146484, 0.00120544]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [5]:
# 검증 폴드의 점수
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [6]:
# 분할기를 사용한 교차 검증
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [7]:
# 10개의 폴드
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [8]:
"""## 하이퍼파라미터 튜닝"""

#그리드 서치
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [9]:
# 가장 검증 점수가 높은 모델로 재훈련
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [10]:
# 최적의 매개변수
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [11]:
# 각 매개변수별 평균 검증 점수
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [12]:
# 제일 큰 값의 인덱스 추출
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index]) # 인덱스의 params에 저장된 매개변수 출력

{'min_impurity_decrease': 0.0001}


In [13]:
# 조금더 복잡한 파라미터 대입
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [14]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [15]:
# 최상의 파라미터 출력
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [16]:
# 최상의 교차 검증 점수 출력
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [18]:
"""## 랜덤 서치"""

from scipy.stats import uniform, randint

# randint() 클래스 : 정수
rgen = randint(0, 10) # 0-10 사이의 범위 랜덤한 10개의 숫자 샘플링
rgen.rvs(10)

array([4, 6, 6, 4, 7, 6, 5, 8, 1, 1])

In [19]:
# 1000개 샘플링
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 85, 100, 103, 108, 106, 106, 101, 122,  83,  86]))

In [20]:
# uniform() 클래스 : 실수
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.90904637, 0.80249585, 0.26737403, 0.9099509 , 0.29562602,
       0.31056811, 0.0460174 , 0.95224889, 0.52999003, 0.08740204])

In [21]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [22]:
# 100번 반복
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f18beae84f0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f18beaebb20>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f18beaead10>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f18beae8f10>},
                   random_state=42)

In [23]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [24]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [25]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


In [26]:
"""# 확인문제"""
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f18beae84f0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f18beaebb20>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f18beaead10>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f18beae8f10>},
                   random_state=42)

In [27]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
